# 10-3-1 Visdom Example

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

## import visdom

![](figs/turn_on_terminal.png)

Jupyter Notebook > Terminal 를 새로 켜서 `python -m visdom.server` 를 입력하세요!

In [4]:
# visdom 서버가 꺼져있으면 Error가 발생한다.
import visdom
vis = visdom.Visdom()

## Text

In [5]:
# text를 실행하는 방법
# env = 'main'은 한 번에 모든 창을 끌 때 사용하기 위함 , 꼭 필요한 요소는 아님
vis.text("Hello, world!",env="main")

'window_39636af7b493a4'

## image

In [6]:

a=torch.randn(3,200,200)
vis.image(a)

'window_39636b6cfd5230'

## images

In [7]:
# image 3개 28x28 생성됨
vis.images(torch.Tensor(3,3,28,28))

'window_39636bdd3ad376'

## example (using MNIST and CIFAR10)

In [8]:
# 시간이 좀 걸립니다.
# cifar 설치 필요
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

100.0%

#### CIFAR10

In [9]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_39636c223db2ce'

#### MNIST

In [10]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_39636c2416a492'

#### Check dataset

In [11]:
# 32개를 한 번에 보는 방법
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32,
                                          shuffle = False)

In [12]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [13]:
# env='main'에 해당하는 것들 한 번에 끄기(지우기)
vis.close(env="main")

''

## Line Plot

In [14]:
# random을로 5개 만듬
# x축을 넣지 않으면, x는 0부터 1사이의 값을 가짐
Y_data = torch.randn(5)
plt = vis.line (Y=Y_data)

In [15]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [16]:
# loss function update할 때, loss는 한 번씩 받아지고, 그 때 update가 이루어진다.
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

# 추가하고자 하는 Y값 , index 값, window 이름, 확장하는 방식으로 line update하겠다는 뜻
vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_39636c85be2102'

### multiple Line on single windows

In [18]:
# 단일 창에서 두 개의 그래프 확인하기
num = torch.Tensor(list(range(0,10)))
print(num.shape)
# view로 index를 바꿈
num = num.view(-1,1)
print(num.shape)
num = torch.cat((num,num),dim=1)
print(num.shape)

# line 2개
plt = vis.line(Y=torch.randn(10,2), X = num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


### Line info

In [19]:
# line에 information은 dictionary로 붙여줌 , 축의 이름을 붙여줄 수 있다.
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [20]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [21]:
# 2개 짜리 넣는 방법
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

## make function for update line

In [22]:
# loss가 update되는 plot 그리기
# (업데이트 할 plot 이름, loss 값, 들어갈 x값)
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [23]:
# plt 창 띄우기
plt = vis.line(Y=torch.Tensor(1).zero_())

# plt라는 plot에 loss 값을 x=i에 append
for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## close the window

In [24]:
vis.close(env="main")

''